In [75]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error

In [76]:
df = pd.read_csv('./PJME_hourly.csv')

In [77]:
df.head()

,Datetime,PJME_MW
0,2002-12-31 01:00:00,26498.0
1,2002-12-31 02:00:00,25147.0
2,2002-12-31 03:00:00,24574.0
3,2002-12-31 04:00:00,24393.0
4,2002-12-31 05:00:00,24860.0


In [78]:
from sklearn.preprocessing import MinMaxScaler

In [79]:
scaler = MinMaxScaler()

In [80]:
import datetime

In [81]:
df.shape

(145366, 2)

In [82]:
df['Datetime'] = pd.to_datetime(df['Datetime'])

In [83]:
df_train = df.loc[0:11600]
df_train.shape

(11601, 2)

In [84]:
df_train.head()

,Datetime,PJME_MW
0,2002-12-31 01:00:00,26498.0
1,2002-12-31 02:00:00,25147.0
2,2002-12-31 03:00:00,24574.0
3,2002-12-31 04:00:00,24393.0
4,2002-12-31 05:00:00,24860.0


In [85]:
df_train.tail()

,Datetime,PJME_MW
11596,2003-09-04 08:00:00,34645.0
11597,2003-09-04 09:00:00,36290.0
11598,2003-09-04 10:00:00,37611.0
11599,2003-09-04 11:00:00,38450.0
11600,2003-09-04 12:00:00,38762.0


In [86]:
df_train.tail()

,Datetime,PJME_MW
11596,2003-09-04 08:00:00,34645.0
11597,2003-09-04 09:00:00,36290.0
11598,2003-09-04 10:00:00,37611.0
11599,2003-09-04 11:00:00,38450.0
11600,2003-09-04 12:00:00,38762.0


In [88]:
df_test = df.loc[11601:]
df_test.shape

(133765, 2)

In [89]:
df_test.head()

,Datetime,PJME_MW
11601,2003-09-04 13:00:00,38828.0
11602,2003-09-04 14:00:00,39017.0
11603,2003-09-04 15:00:00,39015.0
11604,2003-09-04 16:00:00,38984.0
11605,2003-09-04 17:00:00,39082.0


In [92]:
scaler.fit(df_train["PJME_MW"])
scaled_train = scaler.transform(df_train["PJME_MW"])
scaled_test = scaler.transform(df_test["PJME_MW"])

ValueError: Expected 2D array, got 1D array instead:
array=[26498. 25147. 24574. ... 37611. 38450. 38762.].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [36]:
import pandas as pd
import numpy as np

def create_lstm_dataset(df, sequence_length=5):
    # data = []
    df_lstm = pd.DataFrame(columns=['t date', 't-5', 't-4', 't-3', 't-2', 't-1', 't'])
    
    # target_column = df.columns[0]  # Assuming the target column is the first column
    for index in df.index:
        if(index <= 4):
            continue
        new_row = {'t date': df["Datetime"][index], 't-5': df["PJME_MW"][index-5] , 't-4': df["PJME_MW"][index-4], 't-3': df["PJME_MW"][index-3], 't-2': df["PJME_MW"][index-2], 't-1': df["PJME_MW"][index-1], 't': df["PJME_MW"][index]}
        df_row = pd.DataFrame(new_row, index=[0])
        df_lstm = pd.concat([df_lstm, df_row], ignore_index=True)
    
    return df_lstm


# # Example usage:
# # Assuming 'your_dataframe' is your hourly power consumption dataframe
result_df = create_lstm_dataset(df, sequence_length=5)

In [37]:
import tensorflow as tf

In [39]:
result_df.head()

,t date,t-5,t-4,t-3,t-2,t-1,t
0,2002-12-31 06:00:00,26498.0,25147.0,24574.0,24393.0,24860.0,26222.0
1,2002-12-31 07:00:00,25147.0,24574.0,24393.0,24860.0,26222.0,28702.0
2,2002-12-31 08:00:00,24574.0,24393.0,24860.0,26222.0,28702.0,30698.0
3,2002-12-31 09:00:00,24393.0,24860.0,26222.0,28702.0,30698.0,31800.0
4,2002-12-31 10:00:00,24860.0,26222.0,28702.0,30698.0,31800.0,32359.0


In [40]:
result_df = result_df.set_index('t date')

In [41]:
result_df.shape

(145361, 6)

In [ ]:
df.to_csv('')

In [42]:
result_df.head()

,t-5,t-4,t-3,t-2,t-1,t
t date,,,,,,
2002-12-31 06:00:00,26498.0,25147.0,24574.0,24393.0,24860.0,26222.0
2002-12-31 07:00:00,25147.0,24574.0,24393.0,24860.0,26222.0,28702.0
2002-12-31 08:00:00,24574.0,24393.0,24860.0,26222.0,28702.0,30698.0
2002-12-31 09:00:00,24393.0,24860.0,26222.0,28702.0,30698.0,31800.0
2002-12-31 10:00:00,24860.0,26222.0,28702.0,30698.0,31800.0,32359.0


In [ ]:
q_80 = int(len(dates) * .8)
q_90 = int(len(dates) * .9)

dates_train, X_train, y_train = dates[:q_80], X[:q_80], y[:q_80]

dates_val, X_val, y_val = dates[q_80:q_90], X[q_80:q_90], y[q_80:q_90]
dates_test, X_test, y_test = dates[q_90:], X[q_90:], y[q_90:]

plt.plot(dates_train, y_train)
plt.plot(dates_val, y_val)
plt.plot(dates_test, y_test)

plt.legend(['Train', 'Validation', 'Test'])

In [ ]:
X = result_df

In [ ]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32, return_sequences=True)))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(16, return_sequences=False)))
model.add(tf.keras.layers.Dense(1))
model.summary()

In [ ]:
model.compile(loss='mse',
              optimizer="adam",
              metrics=['mean_absolute_error']
              )